# Preprocessing with tensorflow pipelines

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [13]:
data = pd.read_csv('data/autos.csv', encoding='cp1252', index_col=None)

In [15]:
train, test = train_test_split(data, test_size=0.3, random_state=666)
train, val = train_test_split(train, test_size=0.2, random_state=666)

# drop Price outlier
price_desc = train.price.describe()
max_price_outlier = (price_desc['75%'] - price_desc['25%'])*3 + price_desc['75%']
train = train.loc[(train.price < max_price_outlier) & (train.price > 0),]
val = val.loc[(val.price < max_price_outlier) & (val.price > 0),]
# filter year
train = train.loc[(train.yearOfRegistration >= 1864) & (train.yearOfRegistration <= 2017)]
val = val.loc[(val.price < 80000000) & (val.price > 0),]

train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

# save data
train.to_csv('data/train.csv', sep=',', index=False)
val.to_csv('data/val.csv', sep=',', index=False)
test.to_csv('data/test.csv', sep=',', index=False)

In [16]:
# fill NAs

## model, vehicleType, fuelType NAs with "NA"
train.model = train.model.fillna('_NA')
train.vehicleType = train.vehicleType.fillna('_NA')
train.fuelType = train.fuelType.fillna('_NA')

## fill notRepairedDamage NAs with 'nein'
train.notRepairedDamage = train.notRepairedDamage.fillna('nein')

## fill gearbox NAs with most frequent gearbox for model-brand
gearbox_na = train.copy()\
    .loc[train.gearbox.isna() != True,['gearbox','model','brand']]\
    .groupby(['model','brand'], as_index=False)\
    .agg(lambda x:x.value_counts().index[0])
## fill powerps NAs with mean power for model-brand
powerPS_na = train.copy()\
    .loc[train.powerPS > 0,['powerPS','model','brand']]\
    .groupby(['model','brand'], as_index=False)\
    .mean()

gearbox_powerps_na = pd.merge(gearbox_na, powerPS_na, on=['model','brand'], how='outer')

## Save na_encoding
gearbox_powerps_na.to_csv('data/gearbox_powerps_na.csv', sep=',', index=False)

train = pd.merge(train, gearbox_powerps_na, on=['model','brand'], suffixes = ('','_na'), how='left')

train.loc[train.gearbox.isna(),'gearbox'] = train.loc[train.gearbox.isna(),'gearbox_na']
train.loc[train.powerPS.isna(),'powerPS'] = train.loc[train.powerPS.isna(),'powerPS_na']

## case when for a model-brand there is no info of any gearbox
train.loc[train.gearbox.isna(),'gearbox'] = train.gearbox.mode().values

## case when for a model-brand there is no info of any powerPS
train.loc[train.powerPS.isna(),'powerPS'] = train.powerPS.mean()

train = train.drop(['gearbox_na', 'powerPS_na'], axis=1)

In [17]:
# check NA
train.isna().sum()

dateCrawled            0
name                   0
seller                 0
offerType              0
price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dateCreated            0
nrOfPictures           0
postalCode             0
lastSeen               0
dtype: int64

In [295]:
# Transform yearOfRegistration feature
## Data is from 2017
train.yearOfRegistration = 2017 - train.yearOfRegistration

In [296]:
# Normalize numerical features
numerical_features = ['yearOfRegistration', 'powerPS', 'kilometer']

numerical_features_normalization = pd.DataFrame({
    'feature': numerical_features,
    'mean': [np.mean(train[x]) for x in numerical_features],
    'std': [np.std(train[x]) for x in numerical_features]
})

numerical_features_normalization

# save normalization parameters
numerical_features_normalization.to_csv('data/numerical_features_normalization.csv', sep=",", index=False)

In [297]:
numerical_features_normalization

,feature,mean,std
0,yearOfRegistration,13.822326,7.665979
1,powerPS,115.608013,190.277829
2,kilometer,125670.457330,40040.277549


In [23]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, target_name, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    targets = dataframe.pop(target_name)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), targets))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
    return ds

In [24]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size, target_name='price')

In [15]:
def set_numerical_feature(name):
    
    numerical_feature = tf.feature_column.numeric_column(name)
    
    return numerical_feature
    
def set_one_hot_feature(name, data):
    one_hot_feature = tf.feature_column.categorical_column_with_vocabulary_list(name, data[name].unique().tolist())
    one_hot_feature = tf.feature_column.indicator_column(one_hot_feature)
    
    return one_hot_feature

def set_embedding_feature(name, data, dims):
    embedding_feature = tf.feature_column.categorical_column_with_vocabulary_list(name, data[name].unique().tolist())
    embedding_feature = tf.feature_column.embedding_column(embedding_feature, dimension=dims)
    
    return embedding_feature

In [16]:
def feature_columns(data, dataset):
    
    feature_columns = []

    # numeric cols
    for header in ['yearOfRegistration', 'powerPS', 'kilometer']:
        feature_columns.append(set_numerical_feature(header))

    feature_columns.append(set_one_hot_feature('abtest', data))
    
    feature_columns.append(set_embedding_feature('vehicleType', data, 4))
    
    feature_columns.append(set_one_hot_feature('gearbox', data))
    
    feature_columns.append(set_embedding_feature('model', data, 8))
    
    feature_columns.append(set_one_hot_feature('fuelType', data))
    
    feature_columns.append(set_embedding_feature('brand', data, 6))
    
    feature_columns.append(set_one_hot_feature('notRepairedDamage', data))
    
    feature_columns.append(set_embedding_feature('postalCode', data, 10))
    

    feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
    
    return(feature_layer)

In [17]:
f_layer = feature_columns(train, train_ds)

In [326]:
def preprocessing_fn(path, norm_params, na_encoding):
    
    # Load files as pandas df
    data = pd.read_csv(path, index_col=None)
    norm_params = pd.read_csv(norm_params, encoding='cp1252', index_col=None)
    na_encoding = pd.read_csv(na_encoding, encoding='cp1252', index_col=None)
    
    # Fill NAs model, vehicle and fuel with 'NA'
    data.model = data.model.fillna('NA')
    data.vehicleType = data.vehicleType.fillna('NA')
    data.fuelType = data.fuelType.fillna('NA')
    
    ## fill notRepairedDamage NAs with 'nein'
    data.notRepairedDamage = data.notRepairedDamage.fillna('nein')
    
    ## fill gearbox and powerPS NAs with na_encoding
    data = pd.merge(data, na_encoding, on=['model','brand'], suffixes = ('','_na'), how='left')
    data.loc[data.gearbox.isna(),'gearbox'] = data.loc[data.gearbox.isna(),'gearbox_na']
    data.loc[data.powerPS.isna(),'powerPS'] = data.loc[data.powerPS.isna(),'powerPS_na']
    data = data.drop(['gearbox_na', 'powerPS_na'], axis=1)
    
    ## Drop rows where all rows for a model-brand combination are NA
    #data = data.loc[(data.gearbox.isna() == False) & (data.powerPS.isna() == False),]
    
    
    # Numerical features normalization
    for num_col in ['yearOfRegistration', 'powerPS', 'kilometer']:
        if num_col == 'yearOfRegistration':
            data[num_col] = 2017 - data[num_col] # feature as car age, we assume we are in 2017.
        data[num_col] = (data[num_col] - norm_params.loc[norm_params.feature == num_col,'mean'].values)/norm_params.loc[norm_params.feature == num_col,'std'].values
    
    return data

In [327]:
a = preprocessing_fn(path='data/train.csv', norm_params='data/numerical_features_normalization.csv', na_encoding='data/gearbox_powerps_na.csv')

In [328]:
a.gearbox.isna().sum()

2851